Главная задача:
1) Выиграть время до банкротства. (По налогам год-полтора + .5 млн расходов)
- Сколько текущая задолженность.
    - По данным УФНС (на ноябрь 2022) задолженность: 4,5 (без 2 квартала + 1,5). Если убрать доначисления.
        - Смоделировать по текущий день (это неделя).
        - Прикинуть по текущий день (час-два).
2) По возможности выйти за 2.250.
- Возможно сравнивать именно задолженность по требованиям.
    - Узнать, есть ли 4 квартал (по когда?)
- Есть еще неразнесенные документы спорного периода, на 400 тыс.



In [11]:
from importlib import reload
import logging

import pandas as pd

import import_excel, views

reload(logging)
reload(import_excel)
reload(views)

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s:%(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')
pd.set_option('display.max_rows', 1000)

view = views.UFNSView()


/home/mt/PycharmProjects/balanceTax/venv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
2023-04-06 20:06:24 INFO:Загрузка файла завершена. В платежах содержится 65 операций.
2023-04-06 20:06:24 INFO:Загрузка файла завершена. В платежах содержится 326 операций.
/home/mt/PycharmProjects/balanceTax/venv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
2023-04-06 20:06:24 INFO:Загрузка файла завершена. В платежах содержится 637 операций.
2023-04-06 20:06:25 INFO:Загрузка файла завершена. В балансе содержится 816 операций.
2023-04-06 20:06:26 INFO:Загрузка файла завершена. В балансе содержится 1729 операций.
2023-04-06 20:06:27 INFO:Загрузка файла завершена. В балансе содержит

In [13]:
""" Прикинуть в общих числах, задолженность на сейчас

Алгоритм
Налоги[на последнее число операций.] - Берем из УФНС

+ банковские платежи - берем сразу из базы
    + 3543917.07 - Арест и Взыскание в Банке

- начисления 4, 1 квартала.

"""
df = view.df
last_day_operation = df.operation_date.iloc[-1]
tax_balance_last_day = view.model_balance().balance.sum()

view.sber.operation_date.iloc[-1]


datetime.date(2022, 11, 25)